In [1]:
import mysql.connector
from tabulate import tabulate
import pandas as pd
import math
import sys

## Connect to Remote Database
## Insert database information

HOST = "147.46.15.238"
PORT = ""
USER = "DS2024_0024"
PASSWD = "DS2024_0024"
DB = "DS_proj_12"

connection = mysql.connector.connect(
    host=HOST,
    port=7000,
    user=USER,
    passwd=PASSWD,
    db=DB,
    autocommit=True  # to create table permanently
)

cur = connection.cursor(dictionary=True)

## 수정할 필요 없는 함수입니다.
# DO NOT CHANGE INITIAL TABLES IN prj.sql
def get_dump(mysql_con, filename):
    '''
    connect to mysql server using mysql_connector
    load .sql file (filename) to get queries that create tables in an existing database (fma)
    '''
    query = ""
    try:
        with mysql_con.cursor() as cursor:
            for line in open(filename, 'r'):
                if line.strip():
                    line = line.strip()
                    if line[-1] == ";":
                        query += line
                        cursor.execute(query)
                        query = ""
                    else:
                        query += line

    except Warning as warn:
        print(warn)
        sys.exit()


def get_output(query):
    cur.execute(query)
    out = cur.fetchall()
    df = pd.DataFrame(out)
    return df

get_dump(connection, 'prj.sql')

In [2]:
query = 'SELECT item, count(user) AS count\
		 FROM ratings\
		 WHERE rating is not null\
		 GROUP BY item\
		 ORDER BY count DESC\
		 LIMIT 10'

get_output(query)

,item,count
0,28,226
1,22,212
2,54,201
3,203,194
4,59,194
5,38,189
6,232,187
7,101,181
8,83,174
9,51,172


In [54]:
query ='SELECT A.item, ROUND(avg(A.scaled_rating),4) AS avg_rating\
    	FROM (SELECT user, item, (rating-min)/(max-min) AS scaled_rating\
			  FROM ratings AS R NATURAL JOIN (SELECT user,\
											CASE count(rating)\
													WHEN 1 then 0\
											ELSE min(rating)\
											END AS min,\
											max(rating) AS max\
											FROM ratings\
											GROUP BY user) AS S\
			 ) AS A\
        GROUP BY A.item\
		ORDER BY avg_rating DESC'
df = get_output(query)


In [55]:
df

,item,avg_rating
0,316,0.8666
1,203,0.8634
2,439,0.8528
3,69,0.8458
4,4,0.8427
...,...,...
448,451,0.3215
449,450,0.3206
450,58,0.3096
451,63,0.2929


In [56]:
query = 'SHOW tables'
get_output(query)

,Tables_in_DS_proj_12
0,item_similarity
1,ratings
2,user_similarity


## algo 3

### 아이템 유사도

In [130]:
rec_num = 2

In [131]:
query = 'SELECT item_1, sum(sim) AS sim_sum \
		FROM (SELECT item_1, item_2, sim, ROW_NUMBER() OVER(PARTITION BY item_1 ORDER BY sim DESC, item_2 ASC) AS ranking\
			FROM item_similarity) AS A\
        WHERE A.ranking <= ' + str(rec_num) +'\
        GROUP BY item_1'
get_output(query)

,item_1,sim_sum
0,0,1.6
1,1,1.6
2,2,1.6
3,3,1.8
4,4,1.7
...,...,...
448,448,1.6
449,449,1.5
450,450,1.6
451,451,1.4


In [133]:
query2 = 'SELECT I.item_1, I.item_2, ROUND(I.sim/S.sim_sum , 4) AS nor_sim\
		FROM (SELECT item_1, item_2, sim, ROW_NUMBER() OVER(PARTITION BY item_1 ORDER BY sim DESC, item_2 ASC) AS ranking\
			FROM item_similarity) AS I NATURAL JOIN ('+ query +') AS S\
        WHERE I.ranking <= ' + str(rec_num) +''
item_sim = get_output(query2)
item_sim

,item_1,item_2,nor_sim
0,0,3,0.5000
1,0,7,0.5000
2,1,19,0.5000
3,1,29,0.5000
4,2,22,0.5000
...,...,...,...
901,450,39,0.5000
902,451,9,0.5000
903,451,21,0.5000
904,452,1,0.5000


### 아이템-사용자

In [134]:
query = 'SELECT R.item, R.user, T.rate_avg, R.rating\
    	FROM ratings AS R NATURAL JOIN (SELECT item, avg(rating) AS rate_avg\
										FROM ratings\
										GROUP BY item) AS T'

get_output(query)

,item,user,rate_avg,rating
0,0,0,3.364130,4.00
1,1,0,3.990566,5.00
2,2,0,3.526316,5.00
3,3,0,4.036913,4.00
4,4,0,4.379310,4.00
...,...,...,...,...
132271,448,291,3.215686,None
132272,449,291,3.656250,None
132273,450,291,2.571429,None
132274,451,291,2.467742,None


In [135]:
query = 'SELECT R.item, R.user,\
			CASE \
				WHEN R.rating is null THEN T.rate_avg\
			ELSE R.rating\
			END AS rate\
    	FROM ratings AS R NATURAL JOIN (SELECT item, avg(rating) AS rate_avg\
										FROM ratings\
										GROUP BY item) AS T'

item_user = get_output(query)
item_user

,item,user,rate
0,0,0,4.00
1,1,0,5.00
2,2,0,5.00
3,3,0,4.00
4,4,0,4.00
...,...,...,...
132271,448,291,3.215686
132272,449,291,3.656250
132273,450,291,2.571429
132274,451,291,2.467742


### 행렬곱
item_sim , item_user

In [136]:
import pandas as pd

In [145]:
item_user

,item,user,rate
0,0,0,4.00
1,1,0,5.00
2,2,0,5.00
3,3,0,4.00
4,4,0,4.00
...,...,...,...
132271,448,291,3.215686
132272,449,291,3.656250
132273,450,291,2.571429
132274,451,291,2.467742


In [147]:
ii_pivot = pd.pivot_table(item_sim, index= 'item_1', columns= 'item_2', values= 'nor_sim').fillna(0)
ii_pivot

C:\Users\삼성\AppData\Local\Temp\ipykernel_5544\952809161.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ii_pivot = pd.pivot_table(item_sim, index= 'item_1', columns= 'item_2', values= 'nor_sim').fillna(0)


item_2,0,1,2,3,4,5,6,7,8,9,...,417,419,420,422,423,424,425,426,431,434
item_1,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.5000,0.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.4706,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
449,0.0,0.0,0.0,0.4667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
450,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
iu_pivot = pd.pivot_table(item_user, index='item', columns='user', values='rate')
iu_pivot

user,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
item,,,,,,,,,,,,,,,,,,,,,
0,4.0,4.0,3.36413,3.0,3.36413,3.0,3.36413,3.0,3.36413,3.0,...,3.36413,3.36413,5.0,4.0,3.36413,3.36413,3.0,3.36413,3.36413,3.36413
1,5.0,3.990566,3.990566,3.990566,3.990566,3.0,3.990566,3.990566,3.990566,3.990566,...,3.990566,3.990566,3.990566,4.5,3.990566,4.0,3.990566,3.990566,3.990566,3.990566
2,5.0,3.526316,4.0,3.526316,3.526316,2.5,3.526316,3.0,3.526316,3.526316,...,3.526316,3.526316,4.0,3.526316,3.526316,3.526316,3.0,3.526316,2.0,3.526316
3,4.0,4.036913,4.036913,4.036913,5.0,5.0,5.0,5.0,4.036913,4.0,...,5.0,3.0,4.036913,4.5,4.036913,4.0,3.0,4.5,4.036913,4.036913
4,4.0,4.37931,4.37931,4.37931,5.0,5.0,5.0,4.0,4.37931,4.37931,...,5.0,4.37931,4.0,5.0,5.0,4.0,4.37931,4.5,4.37931,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686,...,3.215686,3.215686,3.215686,4.0,3.215686,3.215686,3.215686,3.215686,3.215686,3.215686
449,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,...,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,3.65625,4.0,3.65625,3.65625
450,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,...,2.571429,2.571429,2.571429,2.571429,2.571429,2.571429,3.0,2.571429,2.571429,2.571429
